# House prices regression
- here you can read the Medium blogpost with more explanations -> 
- this is my kaggle profile ->  https://www.kaggle.com/dorianb96

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

### Read the data
- drop the id column for training data, but remember it for the test data
- extract the label (house price) and transform it to log space
- join the training and testing data into a single dataframe for easier transformation with pandas

In [2]:
# read the data 
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# get the training data
x_train_raw = train_data.drop(['Id','SalePrice'],axis=1)
x_test_raw = test_data.drop('Id',axis=1)


# remember the lenghts so we can split the data later
train_length = len(x_train_raw)
test_length = len(x_test_raw)

# concatenate all of the training data into one dataframe so that transformations are easier
all_data = pd.concat([x_train_raw, x_test_raw])


# remember the test data indices only for the final submission 
x_test_ids = test_data['Id']

# get the training data - house prices
# normal prices have a skew so adjust them to log space
y_train = np.log(train_data['SalePrice'])


#display the data
pd.set_option('display.max_columns', 500)
x_test_raw.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0,0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0,0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


### Transforming features 
- missing -> encode 
- categorical -> One hot encoding
- numerical -> scaling

In [3]:
# a nice trick to find out numeric vs categorical features
numerical_features = x_train_raw.columns[x_train_raw.dtypes != 'object']
categorical_features = x_train_raw.columns[x_train_raw.dtypes == 'object']

# encode missing numbers as a special large number
all_data[numerical_features] = all_data[numerical_features].fillna(99999999)

# encode missing data as a special category -> missing
all_data[categorical_features] = all_data[categorical_features].fillna("Missing")

all_data.head(2)

# transform numeric variables 
ss = StandardScaler()
all_data[numerical_features] = ss.fit_transform(all_data[numerical_features])

# transform categorical variables
all_data = pd.get_dummies(data=all_data, columns=categorical_features)


### Re-split the data back to original proportions

In [4]:
# re-split again
x_train = all_data.head(train_length)
x_test = all_data.tail(test_length)

print("Raw features vs features in transformed model:", x_train_raw.shape[1] ,'vs', x_train.shape[1], 'features')

Raw features vs features in transformed model: 79 vs 311 features


### XGBoost regressor 

In [5]:
est = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05, max_depth=3, 
                                max_features='sqrt',min_samples_leaf=15, min_samples_split=10, 
                                loss='huber',random_state = 13)
est.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=15,
             min_samples_split=10, min_weight_fraction_leaf=0.0,
             n_estimators=2000, presort='auto', random_state=13,
             subsample=1.0, verbose=0, warm_start=False)

### Write the submission 
    - you should be able to get around 0.12104 score which would place you into top 1/3 of the scores (~685/2300+)

In [6]:
y_test = np.exp(est.predict(x_test))
result = pd.DataFrame({'Id': x_test_ids.tolist(), 'SalePrice': y_test.ravel()})
result.to_csv('submission.csv',index=False)

### XGBoost Regressors  feature importances

In [7]:
'''
import matplotlib.pyplot as plt
%matplotlib inline


feature_importance = est.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = (np.arange(sorted_idx.shape[0]) + .5 )[-20:]
fig = plt.figure(figsize=(25,10))
ax = fig.add_subplot(111)
plt.barh(pos, feature_importance[sorted_idx][-20:], align='center')
plt.yticks(pos, x_train.columns[sorted_idx][-20:])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()
'''

"\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\n\nfeature_importance = est.feature_importances_\n# make importances relative to max importance\nfeature_importance = 100.0 * (feature_importance / feature_importance.max())\nsorted_idx = np.argsort(feature_importance)\npos = (np.arange(sorted_idx.shape[0]) + .5 )[-20:]\nfig = plt.figure(figsize=(25,10))\nax = fig.add_subplot(111)\nplt.barh(pos, feature_importance[sorted_idx][-20:], align='center')\nplt.yticks(pos, x_train.columns[sorted_idx][-20:])\nplt.xlabel('Relative Importance')\nplt.title('Variable Importance')\nplt.show()\n"

### Building the Keras model
- simple architecture for a regression model
- relu activation function
- mean_squared_logarithmic_error is used as competition metric

In [8]:
"""
from keras.models import Sequential
from keras.layers.core import Dense

model = Sequential()
model.add(Dense(1024, input_dim= x_train.shape[1], init='normal', activation='relu'))
model.add(Dense(256, init='normal', activation='relu'))
model.add(Dense(64, init='normal', activation='relu'))
model.add(Dense(16, init='normal', activation='relu'))
model.add(Dense(1, init='normal'))

model.compile(loss='mean_squared_logarithmic_error', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=1)

#Save the model
from keras.models import load_model

model.save('model_house_prices.h5')  # creates a HDF5 file 'my_model.h5'

"""

"\nfrom keras.models import Sequential\nfrom keras.layers.core import Dense\n\nmodel = Sequential()\nmodel.add(Dense(1024, input_dim= x_train.shape[1], init='normal', activation='relu'))\nmodel.add(Dense(256, init='normal', activation='relu'))\nmodel.add(Dense(64, init='normal', activation='relu'))\nmodel.add(Dense(16, init='normal', activation='relu'))\nmodel.add(Dense(1, init='normal'))\n\nmodel.compile(loss='mean_squared_logarithmic_error', optimizer='adam')\nmodel.fit(x_train, y_train, epochs=100, batch_size=1)\n\n#Save the model\nfrom keras.models import load_model\n\nmodel.save('model_house_prices.h5')  # creates a HDF5 file 'my_model.h5'\n\n"

##  Stacking

In [9]:
'''
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso,ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.ensemble import RandomForestRegressor

model1 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05)
model2 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=3, max_features='sqrt',min_samples_leaf=15, min_samples_split=10, loss='huber')
model3 = GradientBoostingRegressor(n_estimators=1000,max_depth=10)
model4 = GradientBoostingRegressor(n_estimators=25,max_depth=50)
model5 = GradientBoostingRegressor(n_estimators=3000,max_depth=2,max_features='log2')

model6 = RandomForestRegressor(n_estimators = 100,max_features='auto',max_depth=100)
model7 = RandomForestRegressor(n_estimators = 100,max_depth=10,min_samples_leaf=3)
model8 = RandomForestRegressor(n_estimators = 100,max_depth=5,min_samples_leaf=2)
model9 = RandomForestRegressor(n_estimators = 100,max_features='log2', max_depth=2)

model10 = Ridge(random_state=1)
model11 = Lasso(random_state=1)
model12 = SVR(kernel='linear')
model13 = SVR(kernel='rbf',max_iter=10000)

model14 = ElasticNet(alpha = 0.0001, l1_ratio=1, max_iter=10000)
model15 = ElasticNet(alpha =1 , l1_ratio=.01, max_iter=10000)
model16 = LinearRegression()


models = [model1,model2,model3,model4,model5,model6,model7,model8,model9,model10, model11,model12,model13,model14,model15,model16]



'''


# averaging version
'''
Final_labels = np.zeros([1459])
for model in models:
    model = model.fit(x_train, y_train)
    Final_labels += np.exp(model.predict(x_test))
Final_labels = Final_labels / float(18)
'''


'''
best_estimator = [10000,None]


for model in models:
    clf = StackingRegressor(regressors=models, meta_regressor=model)
    scores = cross_val_score(clf, x_train, y_train, cv=5,scoring='neg_mean_squared_error')
    scores = np.sqrt(-scores)
    score = scores.mean()
    if  score < best_estimator[0]:
        best_estimator[0] = score
        best_estimator[1] = clf
    print("Accuracy: %0.3f (+/- %0.3f)" % (score, scores.std() * 2))

        
est = best_estimator[1]
'''


'\nbest_estimator = [10000,None]\n\n\nfor model in models:\n    clf = StackingRegressor(regressors=models, meta_regressor=model)\n    scores = cross_val_score(clf, x_train, y_train, cv=5,scoring=\'neg_mean_squared_error\')\n    scores = np.sqrt(-scores)\n    score = scores.mean()\n    if  score < best_estimator[0]:\n        best_estimator[0] = score\n        best_estimator[1] = clf\n    print("Accuracy: %0.3f (+/- %0.3f)" % (score, scores.std() * 2))\n\n        \nest = best_estimator[1]\n'

## XGBoost regressor Cross validation grid search

In [10]:
'''
from sklearn.model_selection import cross_val_score,GridSearchCV

est = GradientBoostingRegressor(loss='huber')


param_grid = [
  {'n_estimators': [3000],
   'learning_rate': [0.05, 0.01],
    'min_samples_leaf': [3, 6],
    'max_depth': [3, 5], 
    'max_features': ['log2','auto'], 
   'subsample' : [0.7,0.9] }
 ]

gs_cv = GridSearchCV(est, param_grid, scoring='neg_mean_squared_error', n_jobs=4).fit(x_train, y_train)

est = gs_cv.best_estimator
'''

"\nfrom sklearn.model_selection import cross_val_score,GridSearchCV\n\nest = GradientBoostingRegressor(loss='huber')\n\n\nparam_grid = [\n  {'n_estimators': [3000],\n   'learning_rate': [0.05, 0.01],\n    'min_samples_leaf': [3, 6],\n    'max_depth': [3, 5], \n    'max_features': ['log2','auto'], \n   'subsample' : [0.7,0.9] }\n ]\n\ngs_cv = GridSearchCV(est, param_grid, scoring='neg_mean_squared_error', n_jobs=4).fit(x_train, y_train)\n\nest = gs_cv.best_estimator\n"